In [310]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms as transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import torchvision.models as models
from net import *

In [311]:
batch_size = 32
transform = transforms.Compose([
    # Must convert all images to tensors first to be processed.
    transforms.ToTensor()
])

train_set = CIFAR10("data", train=True,transform=transform,download=True)

Files already downloaded and verified


In [312]:
# Create a loader for the training set
train_loader = DataLoader(train_set,batch_size=batch_size,shuffle=True,num_workers=4)

# Load the test set, note that train is set to False
test_set = CIFAR10(root="data", train=False, transform=transform, download=True)

# Create a loder for the test set, note that both shuffle is set to false for the test loader
test_loader = DataLoader(test_set, batch_size=batch_size,shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [313]:
# Create model, optimizer and loss function
num_layers=1
num_channels=32
model = SimpleNet(num_layers=num_layers, num_channels=num_channels, num_classes=10)

# Check if gpu support is available
cuda_avail = torch.cuda.is_available()

# if cuda is available, move the model to the GPU
if cuda_avail:
    model.cuda()

# Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [314]:
def adjust_learning_rate(epoch):
    lr = 0.001

#     if epoch > 180:
#         lr = lr / 1000000
#     elif epoch > 150:
#         lr = lr / 100000
#     elif epoch > 120:
#         lr = lr / 10000
#     elif epoch > 90:
#         lr = lr / 1000
#     elif epoch > 60:
#         lr = lr / 100
#     elif epoch > 30:
#         lr = lr / 10

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [315]:
# Saves a model 
def save_models(epoch):
    torch.save(model.state_dict(), "models/cifar10_{0}layers_{1}channels_epoch{2}.model".format(num_layers, num_channels, epoch))
    print("Checkpoint saved")

# Evaluate performance of a model
def test():
    model.eval()
    test_acc = 0.0
    for i, (images, labels) in enumerate(test_loader):

        if cuda_avail:
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        # Predict classes using images from the test set
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        
        test_acc += torch.sum(prediction == labels.data)

    # Compute the average acc and loss over all 10000 test images
    test_acc = test_acc / 10000

    return test_acc

In [316]:
# Trains a model and saves it if it has the best test accuracy we've seen so far
def train(num_epochs):
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_acc = 0.0
        train_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Move images and labels to gpu if available
            if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            # Clear all accumulated gradients
            optimizer.zero_grad()
            # Predict classes using images from the test set
            outputs = model(images)
            # Compute the loss based on the predictions and actual labels
            loss = loss_fn(outputs, labels)
            # Backpropagate the loss
            loss.backward()

            # Adjust parameters according to the computed gradients
            optimizer.step()
            
            train_loss += loss.cpu().data.item() * images.size(0)
            _, prediction = torch.max(outputs.data, 1)
            
            train_acc += torch.sum(prediction == labels.data)

        # Call the learning rate adjustment function
        adjust_learning_rate(epoch)

        # Compute the average acc and loss over all 50000 training images
        train_acc = train_acc / 50000
        train_loss = train_loss / 50000

        # Evaluate on the test set
        test_acc = test()

        # Save the model if the test acc is greater than our current best
        if test_acc > best_acc:
            save_models(epoch)
            best_acc = test_acc

        # Print the metrics
        print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch, train_acc, train_loss,test_acc))

In [317]:
train(10)

Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/l

KeyboardInterrupt: 